In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
import csv
from matplotlib import pyplot as plt
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [45]:
with open('leanCoords.csv',mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [46]:
def export_landmarks(results,action):
    try:
        keypoints = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)

        with open('leanCoords.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass


In [67]:
# For webcam input:
cap = cv2.VideoCapture('lean.avi')
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.


    k=cv2.waitKey(1)
    if k == 97:
      export_landmarks(results,'left')
    if k == 115:
      export_landmarks(results,'neutral')
    if k == 100:
      export_landmarks(results,'right')
    cv2.imshow('MediaPipe Holistic', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('leanCoords.csv')

In [6]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,neutral,0.543642,0.175827,-0.218345,0.999989,0.549976,0.162153,-0.192544,0.999974,0.554557,...,0.070493,0.750420,0.633604,0.969165,-0.115837,0.970045,0.470120,0.979554,-0.116215,0.964302
1,neutral,0.546412,0.173854,-0.225710,0.999989,0.552340,0.160654,-0.199486,0.999975,0.556228,...,0.084965,0.753505,0.633508,0.971520,-0.100845,0.970999,0.470211,0.979659,-0.101456,0.965443
2,neutral,0.551265,0.172840,-0.290428,0.999989,0.556371,0.159922,-0.264890,0.999974,0.560699,...,0.088947,0.756995,0.634109,0.971622,-0.095475,0.971788,0.470424,0.979685,-0.100368,0.966886
3,neutral,0.554283,0.176627,-0.130254,0.999987,0.560225,0.165404,-0.102089,0.999970,0.564143,...,0.105602,0.692820,0.623076,0.965716,-0.058171,0.962088,0.491070,0.963767,-0.064456,0.955976
4,neutral,0.558957,0.177436,-0.159465,0.999985,0.565267,0.166830,-0.132173,0.999968,0.569156,...,0.124131,0.665324,0.622817,0.966141,-0.027958,0.959388,0.492522,0.967569,-0.050455,0.952429


In [7]:
X = df.drop('class',axis=1) #features
y = df['class'] #target

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1234)

In [9]:
y_test

319    neutral
658      right
842       left
637      right
166    neutral
        ...   
36     neutral
499      right
64     neutral
201    neutral
37     neutral
Name: class, Length: 278, dtype: object

### Train Machine Learning Classification Model

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [11]:
pipelines = {
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [12]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model
    

In [13]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [14]:
fit_models['rc'].predict(X_test)

array(['neutral', 'right', 'left', 'right', 'neutral', 'neutral', 'left',
       'neutral', 'right', 'neutral', 'right', 'neutral', 'right',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'right', 'left', 'right', 'right', 'neutral', 'right', 'neutral',
       'neutral', 'left', 'left', 'neutral', 'neutral', 'right', 'left',
       'right', 'left', 'neutral', 'neutral', 'right', 'neutral',
       'neutral', 'right', 'neutral', 'neutral', 'neutral', 'neutral',
       'right', 'left', 'neutral', 'left', 'right', 'left', 'right',
       'neutral', 'left', 'left', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'right', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'right', 'neutral',
       'neutral', 'right', 'neutral', 'neutral', 'right', 'neutral',
       'neutral', 'right', 'left', 'neutral', 'left', 'neutral', 'right',
       'neutral', 'left', 'right', 'left', 'neutral', 'right', 'right',
       'ri

### Evaluate and Serialize Model

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score # Accuracy Metrics
import pickle

In [16]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test.values,yhat),
    precision_score(y_test.values,yhat,average='weighted'),
    recall_score(y_test.values,yhat,average='weighted'))

lr 1.0 1.0 1.0
rc 1.0 1.0 1.0
rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


In [17]:
yhat = fit_models['rf'].predict(X_test)

In [18]:
yhat[:10]

array(['neutral', 'right', 'left', 'right', 'neutral', 'neutral', 'left',
       'neutral', 'right', 'neutral'], dtype=object)

In [19]:
y_test[:10]

319    neutral
658      right
842       left
637      right
166    neutral
205    neutral
894       left
285    neutral
576      right
155    neutral
Name: class, dtype: object

In [20]:
with open('lean.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)


### Make detections with the Model

In [21]:
with open('lean.pkl','rb') as f:
    model = pickle.load(f)

In [3]:
# For webcam input:
counter = 0
current_stage = ''
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    try:
        row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        X = pd.DataFrame([row],columns=landmarks[1:])
        body_lean_class = model.predict(X)[0]
        body_lean_prob = model.predict_proba(X)[0]
        print(body_lean_class,body_lean_prob)

        if body_lean_class == 'neutral' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'neutral'
            # print("huuuuuu")
        elif body_lean_class == 'left' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'left'
        elif body_lean_class == 'right' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'right'
        
        # Get status box
        cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)

        #Display class
        cv2.putText(image,'CLASS'
            ,(95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,body_lean_class.split(' ')[0]
        , (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        #Display Probability
        cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(round(body_lean_prob[np.argmax(body_lean_prob)],2))
        ,(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        # #Display Probability
        # cv2.putText(image,'COUNT',(180,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        # cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

    except Exception as e:
        pass

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic',image ) #cv2.flip(image, 1)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()